In [1]:
from bs4 import BeautifulSoup
import requests
import time
from urllib.request import urlopen
import ssl
import re
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from os.path import splitext, basename

In [2]:
url='https://www.espncricinfo.com/records/tournament/team-match-results/icc-cricket-world-cup-2023-24-15338'

r= requests.get(url)

In [3]:
r

<Response [200]>

In [4]:
soup= BeautifulSoup(r.text,"html.parser")

table = soup.find("table", class_=lambda x: x and "ds-w-full" in x)


In [5]:
print(table)

<table class="ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide"><thead class="ds-bg-fill-content-alternate ds-text-left"><tr class=""><td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 1</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 2</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Winner</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Margin</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Ground</span></div></td><td class="ds-min-w-max ds-text-right ds-whitespace-nowrap"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Match Date</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-po

In [6]:
header = table.find("thead")

In [7]:
header.text

'Team 1Team 2WinnerMarginGroundMatch DateScorecard'

In [8]:
rows=header.find_all("td")

head=[]
for i in range(len(rows)):
    print(rows[i])

<td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 1</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 2</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Winner</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Margin</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Ground</span></div></td>
<td class="ds-min-w-max ds-text-right ds-whitespace-nowrap"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Match Date</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Scorecard</span></div></td>


In [9]:
rows=header.find_all("td")

head=[]
for i in range(len(rows)):
    
    soup1=rows[i]
    x=soup1.find("span",class_="ds-cursor-pointer")
    print(x)
    head.append(x.text)

<span class="ds-cursor-pointer">Team 1</span>
<span class="ds-cursor-pointer">Team 2</span>
<span class="ds-cursor-pointer">Winner</span>
<span class="ds-cursor-pointer">Margin</span>
<span class="ds-cursor-pointer">Ground</span>
<span class="ds-cursor-pointer">Match Date</span>
<span class="ds-cursor-pointer">Scorecard</span>


In [10]:
head

['Team 1', 'Team 2', 'Winner', 'Margin', 'Ground', 'Match Date', 'Scorecard']

In [15]:
table

<table class="ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide"><thead class="ds-bg-fill-content-alternate ds-text-left"><tr class=""><td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 1</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 2</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Winner</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Margin</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Ground</span></div></td><td class="ds-min-w-max ds-text-right ds-whitespace-nowrap"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Match Date</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-po

In [22]:
body=table.find("tbody")
rows=body.find_all("tr")

data = []
for row in rows:
    temp = []
    # print(row)
    tdata = row.find_all("td", class_="ds-min-w-max")
    for td in tdata:
        temp.append(td.text)

    data.append(temp)


In [23]:
data

[['India',
  'Australia',
  'Australia',
  '6 wickets',
  'Ahmedabad',
  'Nov 19, 2023',
  'ODI # 4705'],
 ['Australia',
  'South Africa',
  'Australia',
  '3 wickets',
  'Eden Gardens',
  'Nov 16, 2023',
  'ODI # 4704'],
 ['India',
  'New Zealand',
  'India',
  '70 runs',
  'Wankhede',
  'Nov 15, 2023',
  'ODI # 4703'],
 ['India',
  'Netherlands',
  'India',
  '160 runs',
  'Bengaluru',
  'Nov 12, 2023',
  'ODI # 4702'],
 ['England',
  'Pakistan',
  'England',
  '93 runs',
  'Eden Gardens',
  'Nov 11, 2023',
  'ODI # 4701'],
 ['Australia',
  'Bangladesh',
  'Australia',
  '8 wickets',
  'Pune',
  'Nov 11, 2023',
  'ODI # 4700'],
 ['Afghanistan',
  'South Africa',
  'South Africa',
  '5 wickets',
  'Ahmedabad',
  'Nov 10, 2023',
  'ODI # 4699'],
 ['New Zealand',
  'Sri Lanka',
  'New Zealand',
  '5 wickets',
  'Bengaluru',
  'Nov 9, 2023',
  'ODI # 4698'],
 ['England',
  'Netherlands',
  'England',
  '160 runs',
  'Pune',
  'Nov 8, 2023',
  'ODI # 4697'],
 ['Afghanistan',
  'Australia'

In [24]:
df=pd.DataFrame(data,columns=head)

In [233]:
df

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard
0,India,Australia,Australia,6 wickets,Ahmedabad,"Nov 19, 2023",ODI # 4705
1,Australia,South Africa,Australia,3 wickets,Eden Gardens,"Nov 16, 2023",ODI # 4704
2,India,New Zealand,India,70 runs,Wankhede,"Nov 15, 2023",ODI # 4703
3,India,Netherlands,India,160 runs,Bengaluru,"Nov 12, 2023",ODI # 4702
4,England,Pakistan,England,93 runs,Eden Gardens,"Nov 11, 2023",ODI # 4701
5,Australia,Bangladesh,Australia,8 wickets,Pune,"Nov 11, 2023",ODI # 4700
6,Afghanistan,South Africa,South Africa,5 wickets,Ahmedabad,"Nov 10, 2023",ODI # 4699
7,New Zealand,Sri Lanka,New Zealand,5 wickets,Bengaluru,"Nov 9, 2023",ODI # 4698
8,England,Netherlands,England,160 runs,Pune,"Nov 8, 2023",ODI # 4697
9,Afghanistan,Australia,Australia,3 wickets,Wankhede,"Nov 7, 2023",ODI # 4696


## Score Card

In [229]:
url1='https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard'

r1= requests.get(url1)

In [230]:
r1

<Response [200]>

In [234]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r1.text, "html.parser")

scorecard = soup.find_all("div","ds-rounded-lg ds-mt-2")

In [ ]:
for i in range(len(scorecard)):
    table1=scorecard[i].find("table","ds-w-full ds-table ds-table-md ds-table-auto  ci-scorecard-table")
    

In [211]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r1.text, "html.parser")

# Use find instead of find_all for the first occurrence
section = soup.find("section", id_="main-container")

In [212]:
from bs4 import BeautifulSoup

# Assuming r1.text contains your HTML content
soup = BeautifulSoup(r1.text, "html.parser")

# Find the section with id "main-container"
main_container = soup.find("section", id="main-container")

In [213]:
section

In [447]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r1.text, "html.parser")

# Use find instead of find_all for the first occurrence
section = soup.find("section", id="main-container")
batting_header=[]


if section:
    # Use find_all instead of find for the second occurrence
    div = section.find_all("div", class_="ds-rounded-lg ds-mt-2")
    temp_header=[]
    
    


    batting_team1 = []
    for i in range(len(div)):
        global test
        test = 0
        temp_header.append(div[i].find("span","ds-capitalize").text)
        # batting_team1.append(div[i].find("span","ds-capitalize").text)
        
        temp_score.append(div[i].find("span","ds-capitalize").text)
        # print(div[i].find("span","ds-capitalize").text)
        bat_table=div[i].find("table","ci-scorecard-table")

        
        body=bat_table.find("tbody")

        # scorecard=body.find_all("td",["ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-flex ds-items-center",\
                                    #   "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-flex ds-items-center ds-border-line-primary ci-scorecard-player-notout"])

        
        scorecard = body.find_all("tr")
        # print(len(scorecard))
        
        
        for j in range(len(scorecard)):
            temp_score=[]
            
            if "ds-hidden" not in scorecard[j].get("class"):
                # print(len(scorecard[i]))
                tdata=scorecard[j].find_all("td")

                if len(tdata)>=7:

                    if tdata[0].find("a")!= None:
                        # print(tdata[0].find("a"))
                        player_name=tdata[0].find("a").get("title")
                        if player_name:

                            temp_score.append((div[i].find("span","ds-capitalize").text))
                            temp_score.append(player_name)
                            temp_score.append(tdata[2].text)
                            temp_score.append(tdata[3].text)
                            temp_score.append(tdata[4].text)
                            temp_score.append(tdata[5].text)
                            temp_score.append(tdata[6].text)
                            temp_score.append(tdata[7].text)
                            print(temp_score)
                            batting_team1.append(temp_score)






            

['India', 'Rohit Sharma', '47', '31', '44', '4', '3', '151.61']
['India', 'Shubman Gill', '4', '7', '21', '0', '0', '57.14']
['India', 'Virat Kohli', '54', '63', '99', '4', '0', '85.71']
['India', 'Shreyas Iyer', '4', '3', '3', '1', '0', '133.33']
['India', 'KL Rahul', '66', '107', '133', '1', '0', '61.68']
['India', 'Ravindra Jadeja', '9', '22', '33', '0', '0', '40.90']
['India', 'Suryakumar Yadav', '18', '28', '57', '1', '0', '64.28']
['India', 'Mohammed Shami', '6', '10', '9', '1', '0', '60.00']
['India', 'Jasprit Bumrah', '1', '3', '5', '0', '0', '33.33']
['India', 'Kuldeep Yadav', '10', '18', '28', '0', '0', '55.55']
['India', 'Mohammed Siraj', '9', '8', '13', '1', '0', '112.50']
['Australia', 'David Warner', '7', '3', '6', '1', '0', '233.33']
['Australia', 'Travis Head', '137', '120', '166', '15', '4', '114.16']
['Australia', 'Mitchell Marsh', '15', '15', '15', '1', '1', '100.00']
['Australia', 'Steven Smith', '4', '9', '11', '1', '0', '44.44']
['Australia', 'Marnus Labuschagne',

In [448]:
batting_team1

[['India', 'Rohit Sharma', '47', '31', '44', '4', '3', '151.61'],
 ['India', 'Shubman Gill', '4', '7', '21', '0', '0', '57.14'],
 ['India', 'Virat Kohli', '54', '63', '99', '4', '0', '85.71'],
 ['India', 'Shreyas Iyer', '4', '3', '3', '1', '0', '133.33'],
 ['India', 'KL Rahul', '66', '107', '133', '1', '0', '61.68'],
 ['India', 'Ravindra Jadeja', '9', '22', '33', '0', '0', '40.90'],
 ['India', 'Suryakumar Yadav', '18', '28', '57', '1', '0', '64.28'],
 ['India', 'Mohammed Shami', '6', '10', '9', '1', '0', '60.00'],
 ['India', 'Jasprit Bumrah', '1', '3', '5', '0', '0', '33.33'],
 ['India', 'Kuldeep Yadav', '10', '18', '28', '0', '0', '55.55'],
 ['India', 'Mohammed Siraj', '9', '8', '13', '1', '0', '112.50'],
 ['Australia', 'David Warner', '7', '3', '6', '1', '0', '233.33'],
 ['Australia', 'Travis Head', '137', '120', '166', '15', '4', '114.16'],
 ['Australia', 'Mitchell Marsh', '15', '15', '15', '1', '1', '100.00'],
 ['Australia', 'Steven Smith', '4', '9', '11', '1', '0', '44.44'],
 ['Au

In [438]:
temp_header

['India', 'Australia']

In [310]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r1.text, "html.parser")

# Use find instead of find_all for the first occurrence
section = soup.find("section", id="main-container")
batting_header=[]
batting_team1 = []

if section:
    # Use find_all instead of find for the second occurrence
    div = section.find_all("div", class_="ds-rounded-lg ds-mt-2")
    temp_header=[]
    

    

    for i in range(len(div)):
        temp_header.append(div[i].find("span","ds-capitalize").text)
        temp_score=[]
        temp_score.append(div[i].find("span","ds-capitalize").text)
        # print(div[i].find("span","ds-capitalize").text)
        bat_table=div[i].find("table","ci-scorecard-table")

        header=bat_table.find_all("th")

        for j in range(len(header)):
            temp_header.append(header[j].text)
        
        body=bat_table.find("tbody")

        # scorecard=body.find_all("td",["ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-flex ds-items-center",\
        #                               "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-flex ds-items-center ds-border-line-primary ci-scorecard-player-notout"])

        scorecard=body.fina_all("tr")
        print(len(scorecard))

        for k in range(len(scorecard)):
            player_name=scorecard[k].find("a").get("title")
            # print(player_name)
            temp_score.append(player_name)

            if scorecard[k].find("td","ds-min-w-max !ds-pl-[100px]"):
                print(scorecard[k].find("td","ds-min-w-max !ds-pl-[100px]").text)
        batting_team1.append(temp_score)





    batting_header.append(temp_header)
    


    
            


    
    

TypeError: 'NoneType' object is not callable

In [307]:
batting_team1

[['India',
  'Rohit Sharma',
  'Shubman Gill',
  'Virat Kohli',
  'Shreyas Iyer',
  'KL Rahul',
  'Ravindra Jadeja',
  'Suryakumar Yadav',
  'Mohammed Shami',
  'Jasprit Bumrah',
  'Kuldeep Yadav',
  'Mohammed Siraj'],
 ['Australia',
  'David Warner',
  'Travis Head',
  'Mitchell Marsh',
  'Steven Smith',
  'Marnus Labuschagne',
  'Glenn Maxwell']]

In [215]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_scorecards(url):
    # ... (rest of the code as provided earlier)

    # Error handling for missing tables
    if not batting_table or not bowling_table:
        return None  # Or raise an appropriate exception

    # ... (extract scores as before)

    return {
        "batting": batting_scorecard,
        "bowling": bowling_scorecard
    }

# Example usage
url = "https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard"
scorecards = get_scorecards(url)

if scorecards:
    batting_df = pd.DataFrame(scorecards["batting"])
    bowling_df = pd.DataFrame(scorecards["bowling"])

    print("Batting DataFrame:")
    print(batting_df)

    print("\nBowling DataFrame:")
    print(bowling_df)
else:
    print("Error: Scorecards not found on the page.")


NameError: name 'batting_table' is not defined

In [171]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r1.text, "html.parser")

# Use find instead of find_all for the first occurrence
section = soup.find("section", id="main-container")
div=section.find("div",class_="ds-rounded-lg ds-mt-2")
country=div.find("span","ds-text-title-xs ds-font-bold ds-capitalize").text
if section:
    # Use find_all instead of find for the second occurrence
    table1 = div.find_all("div", class_="ds-p-0")
    batting = []
    bowling = []
    header1 = []


    table_bat=table1[0].find_all("table")

    header_bat=table_bat[0].find_all("th")
    # print(header_bat)

    for i in range(len(header_bat)):
        if i !=1:
            header1.append(header_bat[i].text)
    table_player=table_bat[0].find_all("tr")

    for i in range(len(table_player)):
        name=table_player[i].find('a', class_='ds-inline-flex ds-items-start ds-leading-none').get_text(strip=True)

    # header_bat=table_bat.find("th")
    # if table_bat:
    #     header_bat=table_bat.find("th")
    #     print(header_bat)




AttributeError: 'NoneType' object has no attribute 'get_text'

In [216]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r1.text, "html.parser")

# Use find instead of find_all for the first occurrence
section = soup.find("section", id="main-container")

# Check if section is found before proceeding
if section:
    div = section.find("div", class_="ds-rounded-lg ds-mt-2")

    # Check if div is found before proceeding
    if div:
        country = div.find("span", "ds-text-title-xs ds-font-bold ds-capitalize").text

        # Use find_all instead of find for the second occurrence
        table1 = div.find_all("div", class_="ds-p-0")
        batting = []
        bowling = []
        header1 = []

        table_bat = table1[0].find_all("table")

        # Check if the table is found before proceeding
        if table_bat:
            header_bat = table_bat[0].find_all("th")

            for i in range(len(header_bat)):
                if i != 1:
                    header1.append(header_bat[i].text)

            table_body=table_bat[0].find("tbody")
            table_player = table_body.find_all("tr")

            for i in range(len(table_player)):
                # print(table_player[i].get("class"))

                if "ds-hidden" not in table_player[i].get("class", []):
                    try:
                        name = table_player[i].find('a', class_='ds-inline-flex ds-items-start ds-leading-none').get_text(strip=True)
                        print(name)
                    except AttributeError:
                        print("Name not found for row", table_player[i])
            
            table_out=table_body.find_all("tr","ds-table-row-compact-top")
            for i in range(len(table_out)):
                print(table_out[i])


                # if "ds-table-row-compact-top" in row.get("class", []):
                #     # This row has the desired class, extract data from it
                #     name_element = row.find('a', class_='ds-inline-flex ds-items-start ds-leading-none')
                    
                #     if name_element:
                #         name = name_element.get_text(strip=True)
                #         print(name)
                #     else:
                #         print("Name not found for row")

            #         # Extract other data from the row if needed
            #         # For example, you can find other elements using row.find("tag_name", class_="class_name")


            # for i in range(len(table_player)):
            #     print(table_player[i])
            #     # Use try-except to handle cases where the element might not be found
            #     try:
            #         name = table_player[i].find('a', class_='ds-inline-flex ds-items-start ds-leading-none').get_text(strip=True)
            #         print(name)
            #     except AttributeError:
            #         print("Name not found for row", table_player[i])


Rohit Sharma(c)
Shubman Gill
Virat Kohli
Shreyas Iyer
KL Rahul†
Ravindra Jadeja
Suryakumar Yadav
Mohammed Shami
Jasprit Bumrah
Kuldeep Yadav
Mohammed Siraj
Name not found for row <tr class="ds-text-tight-s"><td class="ds-min-w-max" colspan="1">Extras</td><td class="ds-min-w-max !ds-pl-[100px]">(lb 3, w 9)</td><td class="ds-min-w-max ds-text-right" colspan="1"><strong>12</strong></td><td class="ds-text-right" colspan="7"></td></tr>
Name not found for row <tr class=""><td class="ds-font-bold ds-bg-fill-content-alternate ds-text-tight-m ds-min-w-max" colspan="1">TOTAL</td><td class="ds-font-bold ds-bg-fill-content-alternate ds-text-tight-m ds-min-w-max ds-flex ds-items-center !ds-pl-[100px]"><span class="ds-font-regular ds-text-tight-s">50 Ov</span><span class="ds-font-regular ds-text-tight-s"> <!-- -->(RR: 4.80)</span><i class="icon-blank_placeholder ds-text-icon" style="font-size:20px"></i></td><td class="ds-font-bold ds-bg-fill-content-alternate ds-text-tight-m ds-min-w-max ds-text-rig

In [186]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r1.text, "html.parser")

# Use find instead of find_all for the first occurrence
section = soup.find("section", id="main-container")

# Check if section is found before proceeding
if section:
    div = section.find("div", class_="ds-rounded-lg ds-mt-2")

    # Check if div is found before proceeding
    if div:
        country = div.find("span", "ds-text-title-xs ds-font-bold ds-capitalize").text

        # Use find_all instead of find for the second occurrence
        table1 = div.find_all("div", class_="ds-p-0")
        batting = []
        bowling = []
        header1 = []

        table_bat = table1[0].find_all("table")

        # Check if the table is found before proceeding
        if table_bat:
            header_bat = table_bat[0].find_all("th")

            for i in range(len(header_bat)):
                if i != 1:
                    header1.append(header_bat[i].text)

            table_body = table_bat[0].find("tbody")
            table_player = table_body.find_all("tr", "ds-table-row-compact-bottom ds-border-none")

            for i in range(len(table_player)):
                name_element = table_player[i].find('a', class_='ds-inline-flex ds-items-start ds-leading-none')
                
                if name_element:
                    name = name_element.get_text(strip=True)
                    print(name)
                else:
                    print("Name not found for row", i)



In [217]:
batting

[]

In [218]:
player_name = soup.find('a', class_='ds-inline-flex ds-items-start ds-leading-none').get_text(strip=True)


In [219]:
name

'Mohammed Siraj'

In [220]:
print(country)

India


In [221]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r1.text, "html.parser")

# Use find instead of find_all for the first occurrence
section = soup.find("section", id="main-container")

if section:
    # Use find_all instead of find for the second occurrence
    table1 = section.find_all("div", class_="ds-rounded-lg ds-mt-2")
    print(table1)
    batting = []
    bowling = []
    header1 = []

    for i in range(len(table1)):
        temp_bat = []
        temp_bowl = []

        # Uncomment the following lines if x is needed
        # x = table1[i].find("span", class_="ds-text-title-xs ds-font-bold ds-capitalize")
        temp_bat.append(x.text)
        temp_bowl.append(x.text)
        print(temp_bat)

        scorecard = table1[i].find_all("table", class_="ds-w-full ds-table ds-table-md ds-table-auto")
        
        if scorecard:
            for j in range(len(scorecard)):
                if j == 0 and scorecard[j]:
                    header_format = scorecard[j].find("thead", class_="ds-bg-fill-content-alternate ds-text-left")

                    if header_format:
                        print(j)
                        header_list = [th.text.strip() for th in header_format.find_all('th') if 'ds-hidden' not in th.get('class', [])]
                        
                        temp_bat.extend(header_list)

            batting.append(temp_bat)

    print(batting)


[<div class="ds-rounded-lg ds-mt-2"><div class="ds-w-full ds-bg-fill-content-prime ds-overflow-hidden ds-rounded-xl ds-border ds-border-line ds-mb-4"><div class="ds-flex ds-px-4 ds-border-b ds-border-line ds-py-3 ds-bg-ui-fill-translucent-hover"><div class="ds-flex ds-flex-col ds-grow ds-justify-center"><span class="ds-text-title-xs ds-font-bold ds-text-typo"><span class="ds-text-title-xs ds-font-bold ds-capitalize">India</span><span class="ds-text-compact-xs ds-font-regular"> <!-- --> <!-- -->(50 ovs maximum)</span></span></div></div><div class="ds-p-0"><table class="ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table"><thead class="ds-bg-fill-content-alternate ds-text-left"><tr class="ds-text-right"><th class="ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-left">BATTING</th><th class="ds-min-w-max"> </th><th class="ds-w-0 ds-whitespace-nowrap ds-min-w-max">R</th><th class="ds-w-0 ds-whitespace-nowrap ds-min-w-max">B</th><th class="ds-w-0 ds-whitespace-nowrap ds-min-w-ma

In [222]:
bowling

[]

In [223]:
temp

[]

In [224]:
print(table)

<table class="ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide"><thead class="ds-bg-fill-content-alternate ds-text-left"><tr class=""><td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 1</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 2</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Winner</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Margin</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Ground</span></div></td><td class="ds-min-w-max ds-text-right ds-whitespace-nowrap"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Match Date</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-po

In [225]:
header = table.find("thead")

In [226]:
header.text

'Team 1Team 2WinnerMarginGroundMatch DateScorecard'

In [227]:
rows=header.find_all("td")

head=[]
for i in range(len(rows)):
    print(rows[i])

<td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 1</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Team 2</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Winner</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Margin</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Ground</span></div></td>
<td class="ds-min-w-max ds-text-right ds-whitespace-nowrap"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Match Date</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Scorecard</span></div></td>


In [228]:
rows=header.find_all("td")

head=[]
for i in range(len(rows)):
    
    soup1=rows[i]
    x=soup1.find("span",class_="ds-cursor-pointer")
    print(x)
    head.append(x.text)

<span class="ds-cursor-pointer">Team 1</span>
<span class="ds-cursor-pointer">Team 2</span>
<span class="ds-cursor-pointer">Winner</span>
<span class="ds-cursor-pointer">Margin</span>
<span class="ds-cursor-pointer">Ground</span>
<span class="ds-cursor-pointer">Match Date</span>
<span class="ds-cursor-pointer">Scorecard</span>
